In [85]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, balanced_accuracy_score,precision_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc, recall_score, confusion_matrix,matthews_corrcoef
import sys 
import pickle

from imblearn.under_sampling import RandomUnderSampler
sys.path.append('/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/')
from RF_atomver import *
from VisUtils import *

In [70]:
data_dir = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/'
inactive_not_in_UNDERtrain['fold'].unique() # 2-5 
data_dir_root = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/undersampler_validation/idea4_dir/'
neks = ['2','3','5','9']
for i, nek in enumerate(neks):
    if nek in ['2','9']: 
        bind_inhib = ['binding', 'inhibition']
    else: 
        bind_inhib = ['binding'] 
    for bi in bind_inhib: 
        if bi == 'binding': 
            this_bi = 'bind'
        else: 
            this_bi = 'inhib' 
        data_dir = f'/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/NEK{nek}/{this_bi}/'
        og_full =  pd.read_csv(f'{data_dir}NEK{nek}_{bi}_moe_scaled_df.csv')
        under = pd.read_csv(f'{data_dir}NEK{nek}_{bi}_moe_UNDER_df.csv')
        print(f'\nNEK{nek} {bi}')
        og_train = og_full[og_full['subset']=='train'] 
        under_train = under[under['subset'] == 'train'] 
        og_train_actives = og_train[og_train['active'] == 1] 
        under_actives = under_train[under_train['active']==1] 
        underactive_not_in_ogactive = under_actives[~under_actives['base_rdkit_smiles'].isin(og_train_actives['base_rdkit_smiles'])]
        underactive_not_in_ogactive
        print(f'under ds: {under_actives.shape}, checking udner actives/all train actives: {underactive_not_in_ogactive.shape}')
        
        under_actives == underactive_not_in_ogactive # true?? 
        
        train_inactive = og_train[og_train['active']==0]
        under_inactive = under_train[under_train['active']==0]
        diff = train_inactive.shape[0] - under_inactive.shape[0]
        print(f'\ntrain inactive {train_inactive.shape}, under train inactive: {under_inactive.shape}\ndifference: train-under train = {diff}')
        inactive_not_in_UNDERtrain = train_inactive[~train_inactive['base_rdkit_smiles'].isin(under_inactive['base_rdkit_smiles'])]
        
        print(f'so, inactives excluded: {inactive_not_in_UNDERtrain.shape}')
        inactive_not_in_UNDERtrain.to_csv(f'{data_dir_root}NEK{nek}_{bi}_moe_excluded_by_RUS.csv', index=False)
        


NEK2 binding
under ds: (45, 311), checking udner actives/all train actives: (45, 311)

train inactive (1080, 311), under train inactive: (45, 311)
difference: train-under train = 1035
so, inactives excluded: (1035, 311)

NEK2 inhibition
under ds: (112, 311), checking udner actives/all train actives: (112, 311)

train inactive (1512, 311), under train inactive: (112, 311)
difference: train-under train = 1400
so, inactives excluded: (1400, 311)

NEK3 binding
under ds: (64, 311), checking udner actives/all train actives: (64, 311)

train inactive (1058, 311), under train inactive: (64, 311)
difference: train-under train = 994
so, inactives excluded: (994, 311)

NEK5 binding
under ds: (77, 311), checking udner actives/all train actives: (77, 311)

train inactive (912, 311), under train inactive: (77, 311)
difference: train-under train = 835
so, inactives excluded: (835, 311)

NEK9 binding
under ds: (48, 311), checking udner actives/all train actives: (48, 311)

train inactive (1078, 311),

In [83]:
def add_cm2(df): 
    true_labels = df['y'] 
    predictions = df['prediction']
    cm = confusion_matrix(true_labels, predictions )
    cm_flattened = cm.flatten().tolist()
    df['cm'] = [cm_flattened]* len(df)
    df['prediction_type'] = df.apply(lambda x: prediction_type(x['y'], x['prediction']), axis=1)
    
    df['f1'] = f1_score(df['y'], df['prediction'])
    # df['ROC-AUC'] = roc_auc_score(df['y'], df['prediction'])
    df['ROC-AUC'] ='NA'
    df['MCC'] = matthews_corrcoef(df['y'], df['prediction'])
    df['Balanced Accuracy'] = balanced_accuracy_score(df['y'], df['prediction'])
    return df
        

# Run test this on the original models 

In [86]:
model_dir = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/RF/RF_pickles/'
data_dir = f'/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/undersampler_validation/idea4_dir/excluded_inactives/'
drop_cols = ['subset', 'compound_id', 'base_rdkit_smiles', 'fold', 'active'] 
for i, nek in enumerate(neks):
    if nek in ['2','9']: 
        bind_inhib = ['binding', 'inhibition']
    else: 
        bind_inhib = ['binding'] 
    for bi in bind_inhib: 
        if bi == 'binding': 
            this_bi = 'bind'
        else: 
            this_bi = 'inhib' 
        df = pd.read_csv(f'{data_dir}NEK{nek}_{bi}_moe_excluded_by_RUS.csv')
        true_values = df['active'] 
        data = df.drop(columns=drop_cols)
        model_name = f'NEK{nek}_{bi}_moe_UNDER_RF.pkl'
        
        with open(f'{model_dir}{model_name}', 'rb') as f: 
            model = pickle.load(f)
        results_df = gather_rf_results(model,data.to_numpy(), true_values.to_numpy())
        results_df['model'] = f'NEK{nek}_{bi}_moe_UNDER_RF'
        results_df = add_cm2(results_df)
        results_df['feat_type'] = 'moe'
        results_df['strategy'] = 'UNDER_excluded_by_RUS'
        results_df['NEK'] = f'NEK{nek}_{bi}'
        results_df['RF_type'] = 'RF'
        results_df.to_csv(f'{data_dir}NEK{nek}_{bi}_moe_UNDER_RF_train_results_excluded_by_RUS.csv',index=False) 

/Users/jayceepang/miniconda3/envs/atom2024/lib/python3.9/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/jayceepang/miniconda3/envs/atom2024/lib/python3.9/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/jayceepang/miniconda3/envs/atom2024/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: 

In [89]:
results = [] 
metric_cols = ['accuracy', 'precision', 'recall', 'specificity','TN', 'FN', 'FP', 'TP','f1', 'ROC-AUC', 'MCC', 'Balanced Accuracy',
       'model', 'cm', 'prediction_type', 'NEK', 'feat_type', 'strategy','RF_type']

for i, nek in enumerate(neks):
    if nek in ['2','9']: 
        bind_inhib = ['binding', 'inhibition']
    else: 
        bind_inhib = ['binding'] 
    for bi in bind_inhib: 
        if bi == 'binding': 
            this_bi = 'bind'
        else: 
            this_bi = 'inhib' 
        result_df = pd.read_csv(f'{data_dir}NEK{nek}_{bi}_moe_UNDER_RF_train_results_excluded_by_RUS.csv')
        results.append(result_df.iloc[[0]][metric_cols].values.flatten())
      

In [91]:
results_df =  pd.DataFrame(results,columns=metric_cols)
results_df['model'] = results_df['model'].str.replace('scaled', 'raw')
results_df['strategy'] =results_df['strategy'].str.replace('scaled', 'raw')
results_df['strategy'] = results_df['strategy'].str.replace('scaled', 'raw')
results_df.to_csv(data_dir+'RF_results_from_train_excluded_by_RUS.csv', index=False) 
results_df['modeling_type'] = 'RF' 
results_df['set'] = 'train_RUS_excluded'

# do a comparison of old and other undersampled 

In [112]:
original_results = pd.read_excel('/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/general_NEK/GP_GPmatern_RF_RFGS_results_all_sets.xlsx')
original_results = original_results[(original_results['set']=='original') & (original_results['modeling_type'] == 'RF') & (original_results['feat_type'] == 'moe')]
only_under = original_results[(original_results['strategy'] == 'UNDER') &(original_results['RF_type'] == 'RF')]
all_under_rf = pd.concat([results_df, only_under])

comparison = all_under_rf[['NEK', 'set','cm', 'recall', 'specificity']]
comparison = comparison.sort_values('NEK')
comparison

,NEK,set,cm,recall,specificity
0,NEK2_binding,train_RUS_excluded,"[734, 301, 0, 0]",0.000000,0.709179
156,NEK2_binding,original,"[191, 80, 6, 6]",0.500000,0.704797
1,NEK2_inhibition,train_RUS_excluded,"[1143, 257, 0, 0]",0.000000,0.816429
188,NEK2_inhibition,original,"[302, 78, 4, 24]",0.857143,0.794737
2,NEK3_binding,train_RUS_excluded,"[570, 424, 0, 0]",0.000000,0.573441
220,NEK3_binding,original,"[135, 130, 3, 14]",0.823529,0.509434
3,NEK5_binding,train_RUS_excluded,"[622, 213, 0, 0]",0.000000,0.744910
252,NEK5_binding,original,"[161, 67, 3, 17]",0.850000,0.706140
4,NEK9_binding,train_RUS_excluded,"[694, 336, 0, 0]",0.000000,0.673786
284,NEK9_binding,original,"[177, 93, 4, 9]",0.692308,0.655556


In [20]:
def scaled_df_with_folds(file_path, file_name, fold):
    """Standard Scalar to normalize original dataset
    Scale data, put back feature names (including ID columns, add subset (train / test) column
    file_path: directory where file is located 
    file_name: should be NEK#_1_uM_min_50_pct_(binding/inhibition)_5fold_random_imbalanced.csv
    returns: final scaled dataframe
    """
    df_original = pd.read_csv(file_path+file_name)
    print(f'original shape: {df_original.shape}')
    df_original = df_original.drop_duplicates(subset='base_rdkit_smiles')
    print(f'remove duplicates: {df_original.shape}')
    # display(df_original)
    original_cols = df_original.columns 
    og_cols = original_cols.to_list() 
    feature_cols = og_cols[3:-1] 
    id_col_names = og_cols[0:2]
    id_cols = df_original[id_col_names]
    fold_col = df_original['fold']
    true_labels = df_original['active']
    # print(f'all cols: {original_cols.shape}, features: {len(feature_cols)}, id: {id_col_names}')
    
    train_df = df_original[df_original['fold']!=fold]
    test_df = df_original[df_original['fold']==fold]
    trainX_df = train_df[feature_cols]
    testX_df = test_df[feature_cols]
    trainy_df = train_df['active']
    testy_df = test_df['active']
    
    train_id_df = train_df[id_col_names]
    test_id_df = test_df[id_col_names]
    train_fold = train_df['fold']
    test_fold = test_df['fold']
    
    x_df = pd.concat([trainX_df, testX_df])
    scaling=StandardScaler()
    scaling.fit(x_df) 
    
    scaled_data=scaling.transform(x_df)
    trainX_scaled = scaling.transform(trainX_df)
    testX_scaled = scaling.transform(testX_df) 
    print(f'train X: {trainX_scaled.shape}, testX: {testX_scaled.shape}')
    trainX_final = pd.DataFrame(trainX_scaled, columns=feature_cols)
    trainX_final['subset'] = 'train'
    
    testX_final = pd.DataFrame(testX_scaled, columns=feature_cols)
    testX_final['subset']='test'
    trainX_final[id_col_names] = train_id_df.reset_index(drop=True)
    testX_final[id_col_names] = test_id_df.reset_index(drop=True)
    trainX_final['fold'] = train_fold.reset_index(drop=True)
    testX_final['fold'] = test_fold.reset_index(drop=True)
    
    trainX_final['active'] = trainy_df.reset_index(drop=True)
    testX_final['active'] = testy_df.reset_index(drop=True)

    final_df = pd.concat([trainX_final, testX_final], ignore_index=True)
    return final_df
    

# take out all samples removed by undersampler (from folds2-5) and test them

In [22]:
data_dir = '/Users/jayceepang/msse/capstone/data/NEK_ATOM_data/'
this_dir = '/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/undersampler_validation/idea4_dir/'
neks = ['2','3','5','9']
for i, nek in enumerate(neks):
    if nek in ['2','9']: 
        bind_inhib = ['binding', 'inhibition']
    else: 
        bind_inhib = ['binding'] 
    for bi in bind_inhib: 
        if bi == 'binding': 
            this_bi = 'bind'
        else: 
            this_bi = 'inhib' 
        nek_path= f'{data_dir}NEK{nek}/'
        assay_file = f'NEK{nek}_1_uM_min_50_pct_{bi}_5fold_random_imbalanced.csv'
        final_df = scaled_df_with_folds(nek_path,assay_file, 'fold2')
        
        final_csv_name = f'{this_dir}NEK{nek}_{bi}_moe_scaled_df_fold2.csv'
        final_df.to_csv(final_csv_name, index=False)

2 binding
original shape: (1408, 310)
remove duplicates: (1408, 310)
train X: (1126, 306), testX: (282, 306)
2 inhibition
original shape: (2044, 310)
remove duplicates: (2032, 310)
train X: (1628, 306), testX: (404, 306)
3 binding
original shape: (1404, 310)
remove duplicates: (1404, 310)
train X: (1123, 306), testX: (281, 306)
5 binding
original shape: (1237, 310)
remove duplicates: (1237, 310)
train X: (989, 306), testX: (248, 306)
9 binding
original shape: (1409, 310)
remove duplicates: (1409, 310)
train X: (1127, 306), testX: (282, 306)
9 inhibition
original shape: (393, 310)
remove duplicates: (393, 310)
train X: (314, 306), testX: (79, 306)
